In [1]:
import sys
sys.path.append('/home/lorenz/github/cost-optimal-model/')


In [2]:
import random
import math
from models.scripts.batch_scheduling import Scheduler
from models.utils import calc_cost

def simulate_annealing(instances, queries, kmax, seed):
    random.seed(a=seed)
    scheduler = Scheduler(instances)

    for query in queries:
        results = scheduler.calc_time(query)
        results = calc_cost([results])
        query["model_results"] = results[0]

    current_cost, max_cost = setup_initial_state(queries, scheduler)
    t0 = current_cost - max_cost

    for k in range(0, kmax):
        print(current_cost)
        t = temperature(k, t0)
        query, provision, id = neighbour(queries, scheduler)
        new_cost = neighbour_cost(current_cost, query, provision, id, scheduler)
        if acceptance_probability(current_cost, new_cost, t) >= random.random():
            apply_change(query, provision, id, scheduler)
            current_cost = new_cost
    print(current_cost)
    return current_state(queries)


def setup_initial_state(queries, scheduler):
    total_cost = 0
    max_cost = 0
    for query in queries:
        sample = query["model_results"].sample(1)
        type_id = sample.index[0]
        instance_id = scheduler.schedule_new_instance(type_id)
        query["instance_id"] = instance_id
        query["type_id"] = type_id

        runtime = query["model_results"].loc[type_id, "stat_time_sum"]
        query_cost = scheduler.schedule_query_cost_new_instance(type_id, runtime)
        scheduler.schedule_query(query, instance_id, runtime)

        total_cost += query_cost
        if query_cost > max_cost:
            max_cost = query_cost
    return total_cost, max_cost

def temperature(k, t0):
    return t0 / (1 + k)

def neighbour(queries, scheduler):
    query = random.choice(queries)
    provisioned_instances = scheduler.suitable_provisioned_instances(query)
    instance_types = scheduler.suitable_instance_types(query)
    index = random.randrange(len(provisioned_instances.index) + 1)
    if index < len(provisioned_instances.index):
        return query, False, provisioned_instances.index[index]
    else:
        index = random.randrange(len(instance_types.index))
        return query, True, instance_types.index[index]

def neighbour_cost(cost, query, provision, id, scheduler):
    if provision:
        new_runtime = query["model_results"].loc[id, "stat_time_sum"]
        cost += scheduler.schedule_query_cost_new_instance(id, new_runtime)
        old_runtime = query["model_results"].loc[query["type_id"], "stat_time_sum"]
        cost += scheduler.unschedule_query_cost(query["instance_id"], old_runtime)
    else:
        new_type_id = scheduler.provisioned_instances.loc[id, "id"]
        new_runtime = query["model_results"].loc[new_type_id, "stat_time_sum"]
        cost += scheduler.schedule_query_cost_existing_instance(id, new_runtime)
        old_runtime = query["model_results"].loc[query["type_id"], "stat_time_sum"]
        cost += scheduler.unschedule_query_cost(query["instance_id"], old_runtime)
    return cost

def acceptance_probability(cost, new_cost, t):
    if new_cost < cost:
        return 1
    return math.exp(-(new_cost - cost)/t)

def apply_change(query, provision, id, scheduler):
    old_instance_id = query["instance_id"]
    old_tpe_id = query["type_id"]
    if provision:
        runtime = query["model_results"].loc[id, "stat_time_sum"]
        instance_id = scheduler.schedule_new_instance(id)
        scheduler.schedule_query(query, instance_id, runtime)
        query["instance_id"] = instance_id
        query["type_id"] = id
    else:
        type_id = scheduler.provisioned_instances.loc[id, "id"]
        runtime = query["model_results"].loc[type_id, "stat_time_sum"]
        scheduler.schedule_query(query, id, runtime)
        query["instance_id"] = id
        query["type_id"] = type_id
    old_runtime = query["model_results"].loc[old_tpe_id, "stat_time_sum"]
    scheduler.unschedule_query(query, old_instance_id, old_runtime)

def current_state(queries):
    for query in queries:
        print("instance type: " + query["type_id"])
        print("instance id: " + query["instance_id"])
    return "done"

In [3]:

from preprocessing.instances import instSet_transform
from models.const import QUERIES


if __name__ == '__main__':
    instances = instSet_transform()
    result = simulate_annealing(instances, QUERIES, 100, 0)
    print(result)


9269075.88432
9088707.3072
8838189.99552
8127614.1388799995
8111517.361919998
8111517.361919998
8355608.969279998
8355608.969279998
8202637.777919998
8099773.943039999
7611757.170239999
7618182.8745599985
2567400.031679998
2497812.139199998
2497812.139199998
2247294.8275199984
2057192.7201599989
2057192.7201599989
2057192.7201599989
2057192.7201599989
627958.1087999996
627958.1087999996
627958.1087999996
530782.2465599995
530782.2465599995
544864.3905599995
642040.2527999993
642040.2527999993
642040.2527999993
642040.2527999993
519036.1252799991
369094.18943999906
369094.18943999906
369094.18943999906
369094.18943999906
369094.18943999906
369094.18943999906
369094.18943999906
369094.18943999906
369094.18943999906
456578.349599999
456578.349599999
524642.7359999991
369094.18943999906
218152.46783999907
218152.46783999907
218152.46783999907
115369.9948799992
115369.9948799992
115369.9948799992
115369.9948799992
115369.9948799992
115369.9948799992
115369.9948799992
115369.9948799992
11536